In [12]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import cell2location
import scvi
import torch
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.backends.backend_pdf as mpdf
import warnings
sys.path.append("/home/biolab/Projects/LAMs_wd/scripts")
warnings.filterwarnings('ignore')
data_type = 'float32'
from visium_qc_and_visualisation import read_and_qc
from config import lung_config

In [13]:
o_sig_matrix = pd.read_csv("/home/biolab/Projects/LAMs_wd/data/signature_adeno_matrix_24_cell_type_renamed.csv", index_col=0)
o_sig_matrix

,Endothelial_cell,other,cDC2,Alveolar_cell_type_2,transitional_club/AT2,Macrophage,Alveolar_cell_type_1,Macrophage_alveolar,Stromal,NK_cell,...,T_cell_CD4,T_cell_regulatory,Mast_cell,pDC,B_cell,cDC1,Ciliated,Monocyte,DC_mature,Neutrophils
ABT1,1.0,63.551991,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0,...,104.614947,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000
NFATC1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0,...,1.000000,1.0,140.215477,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000
PLCD1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0,...,1.000000,1.0,1.000000,1.000000,1.0,106.246914,1.0,1.000000,1.000000,1.000000
NTRK1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0,...,1.000000,1.0,143.735216,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000
CRABP2,1.0,1.000000,1.000000,1.000000,281.739377,1.000000,1.000000,1.00000,109.695136,1.0,...,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TFPI2,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0,...,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.000000,240.841402,1.000000
AGER,1.0,1.000000,1.000000,204.844608,1.000000,1.000000,7757.023824,1.00000,1.000000,1.0,...,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000
LIF,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0,...,1.000000,1.0,263.299665,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000
KDM2B,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0,...,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.000000,256.565968,1.000000


Run Cell2location

In [24]:
def run_cell2loc(ifolder, environment):
    seed = 42
    torch.manual_seed(seed)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    outDir = f"/home/biolab/Projects/LAMs_wd/results/cell2location/spatial_model_SC_Y_sig_matrix_adeno_LUAD_patients/"
    regDir = f"/home/biolab/Projects/LAMs_wd/data/E-MTAB-13530/{ifolder}/cell2location/regression_model/"
    import os
    os.makedirs(outDir + environment, exist_ok=True)

    # Pick up the scRNA-seq gene expression model
#     adata_sc_model = sc.read_h5ad(f"{regDir}{environment}/sc.h5ad")
#     adata_sc_model = sc.read_h5ad("/media/mllab/SSD/ST/lung-master/SpatialTranscriptomics/cell2location/regression_model/Tumour/sc.h5ad")
    
#     mod = cell2location.models.RegressionModel.load(f"{regDir}{environment}", adata_sc_model)
#     adata_sc_model = mod.export_posterior(adata_sc_model, sample_kwargs={'num_samples': 1000, 'batch_size': 2500})
#     adata_sc_model.write(f"{regDir}{environment}/sc.h5ad")

    inf_aver = None
    # Export estimated expression signature in each cluster
    
    inf_aver = o_sig_matrix.copy()

    # Scale up by average sample scaling factor. This corrects for sequencing depth
    # inf_aver = inf_aver * adata_sc_model.uns["mod"]["post_sample_means"]["sample_scaling"].mean() # Need to check if this is still necessary

    # Pick up Visium
    sample_names = lung_config[environment]["spatial"]
    print(sample_names)
    #inDir = "/lustre/scratch118/opentargets/opentargets/OTAR2060/nelson/data/lung/visium/samples/{}/".format(environment.lower())
    inDir = "/home/biolab/Projects/LAMs_wd/data/E-TAMB-13530/"
    slides = [read_and_qc(sample_name, path=inDir) for sample_name in sample_names]
    adata_vis = slides[0].concatenate(slides[1:], batch_key="sample", uns_merge="unique", batch_categories=sample_names, index_unique=None)
    # Mitochondria-encoded (MT) genes should be removed for spatial mapping
    # Find mitochondria-encoded (MT) genes
    adata_vis.var["MT_gene"] = [gene.startswith("MT-") for gene in adata_vis.var["SYMBOL"]]
    # Remove MT genes for spatial mapping (keeping their counts in the object)
    adata_vis.obsm["MT"] = adata_vis[:, adata_vis.var["MT_gene"].values].X.toarray()
    adata_vis = adata_vis[:, ~adata_vis.var["MT_gene"].values]
    # Reset the index for compatibility with the scRNA-seq counts
    adata_vis.var.set_index("SYMBOL", inplace=True)

    # Find shared genes and subset both anndata and reference signatures
    adata_vis.var_names_make_unique()
    
    inf_aver_index = np.array(inf_aver.index, dtype=str)
    adata_vis_var_names = np.array(adata_vis.var_names, dtype=str)
    intersect = np.intersect1d(adata_vis_var_names, inf_aver_index)
    #intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
    #adata_vis = adata_vis[:, intersect] # This doesn't work -- think there's some incompatibility between ScanPy and AnnData versions. Use the line below instead
    adata_vis = adata_vis[:, adata_vis.var.index.isin(intersect)].copy()
    inf_aver = inf_aver.loc[intersect, :]
    # Reindex for cell2location
    inf_aver = inf_aver.reindex(adata_vis.var.index)

    # Save on memory
#     del adata_sc_model
#     del mod

    # Training cell2location
    cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key="sample")
    mod = cell2location.models.Cell2location(adata_vis,
                                             cell_state_df=inf_aver,
                                             detection_alpha=200, # Controls the normalisation of the within-experiment variation of the RNA detection (default value for now)
                                             N_cells_per_location=8
                                             )
    '''
                                             **model_kwargs={
                                                # Prior on the number of cells, cell types and co-located groups. Hyperparameter inputs go here
                                                "cell_number_prior": {
                                                    # - N - the expected number of cells per location:
                                                    "cells_per_spot": 8,
                                                    # - A - the expected number of cell types per location:
                                                    "factors_per_spot": 9,
                                                    # - Y - the expected number of co-located cell type groups per location
                                                    "combs_per_spot": 5
                                                },
                                                # Prior beliefs on the sensitivity of spatial technology:
                                                "gene_level_prior": {
                                                    # Prior on the mean
                                                    "mean": 0.5,
                                                    # Prior on standard deviation
                                                    # A good choice of this value should be at least 2 times lower that the mean
                                                    "sd": 0.15
                                                }
                                             }
                                             )
    '''

    # Try 2500 batches initially, rather than training on the entire data-set
    mod.train(batch_size=2500, train_size=1, max_epochs=3000)
    mod.save(f"{outDir}{environment}", overwrite=True)
    adata_vis = mod.export_posterior(adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': 2500})
    adata_vis.write(f"{outDir}{environment}/sp_our_sig_matrix_all_cts_{environment}.h5ad")

    train_performance = mod.history["elbo_train"]
    train_performance.plot(logy=True)
    plt.xlabel("Training epochs")
    plt.ylabel("-ELBO loss")
    plt.title(f"Lung ({environment})")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.savefig(f"{outDir}/ELBO_Cell2location_{environment}.png")
    plt.close()
    

In [25]:
run_cell2loc("dataset", "Background")

['spaceranger110_count_40751_OTAR_LNGsp10782317_GRCh38-2020-A', 'spaceranger110_count_40612_OTAR_LNGsp10782311_GRCh38-2020-A', 'spaceranger110_count_40952_OTAR_LNGsp10922370_GRCh38-2020-A', 'spaceranger110_count_40951_OTAR_LNGsp10922368_GRCh38-2020-A', 'spaceranger110_count_40751_OTAR_LNGsp10782316_GRCh38-2020-A', 'spaceranger110_count_40952_OTAR_LNGsp10922371_GRCh38-2020-A', 'spaceranger110_count_40612_OTAR_LNGsp10782310_GRCh38-2020-A', 'spaceranger110_count_40951_OTAR_LNGsp10922367_GRCh38-2020-A', 'spaceranger110_count_40952_OTAR_LNGsp10922369_GRCh38-2020-A', 'spaceranger110_count_40952_OTAR_LNGsp10922372_GRCh38-2020-A']


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 3000/3000: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [41:45<00:00,  1.22it/s, v_num=1, elbo_train=1.02e+7]

`Trainer.fit` stopped: `max_epochs=3000` reached.


Sampling global variables, sample: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 999/999 [00:13<00:00, 75.24it/s]
